In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn.metrics import average_precision_score, mean_squared_error, mean_squared_log_error
import swifter
from pytz import timezone
import jpholiday
from datetime import datetime
import joblib
import swifter

pd.options.display.max_columns = 100

from tqdm.notebook import tqdm

import os
from logzero import logger
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import models, transforms
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, QuantileTransformer

from utils import StratifiedGroupKFold

In [2]:
train, test, groups = joblib.load('../output/features.joblib')

In [3]:
X_train = train.drop(['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'], axis=1)
#y_train = train['Global_Sales'].copy()
y_train = np.log1p(train['Global_Sales'])
X_test = test.drop(['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'], axis=1).copy()

In [4]:
category_cols = X_train.columns[X_train.dtypes=='category']
for col in category_cols:
    X_train[col] = X_train[col].astype(object)
    X_test[col] = X_test[col].astype(object)
object_cols = X_train.columns[X_train.dtypes=='object']

In [5]:
for col in tqdm(object_cols):
    X_train[col] = X_train[col].astype(object).astype(str)
    X_test[col] = X_test[col].astype(object).astype(str)
    le = LabelEncoder()
    le.fit(X_train[col])
    le_dict = dict(zip(le.classes_, le.transform(le.classes_)))
    X_train[col] = X_train[col].swifter.apply(lambda x: le_dict.get(x, np.nan))
    X_test[col] = X_test[col].swifter.apply(lambda x: le_dict.get(x, np.nan))

In [6]:
class NetRegressor(nn.Module):
    def __init__(self, in_features, out_features, bias=True, p=0.5):
        super(NetRegressor, self).__init__()
        
        out1 = 128 # 128
        out2 = 64 # 64
        self.regressor = nn.Sequential(
            nn.Linear(in_features, out1),
            nn.BatchNorm1d(out1),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(out1, out2),
            nn.BatchNorm1d(out2),
            nn.ReLU(),
            nn.Linear(out2, 1)
        )
    
    def forward(self, x):
        output = self.regressor(x)
        return output

class EarlyStopping:
    """
    Early stops the training if validation loss doesn't improve after a given patience.
    based on: https://github.com/Bjarten/early-stopping-pytorch
    """
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            logger.info(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            logger.info(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss

        
class MLPRegressor:
    def __init__(self, net, X_train, y_train, X_valid, y_valid, X_test,
                 batch_size=32, num_labels=2, num_epochs=100, bias=True):
        super(MLPRegressor, self).__init__()
        self.net = net
        self.X_train = X_train.values
        self.y_train = y_train.values
        self.X_valid = X_valid.values
        self.y_valid = y_valid.values
        self.X_test = X_test.values
        self.batch_size = batch_size
        self.num_labels = num_labels
        in_features = X_train.shape[1]
        out_features = num_labels
        self.num_epochs = num_epochs

        
        # create dataloaders
        self.train_dataloader = self.train_dataloader()
        self.valid_dataloader = self.valid_dataloader()
        self.test_dataloader = self.test_dataloader()
        self.dataloaders_dict = {'train': self.train_dataloader, 'valid': self.valid_dataloader}
    
    def criterion(self):
        return  nn.MSELoss()

    def optimizer(self):
        return optim.Adam(params=self.net.parameters(), lr=1e-3)
    
    def scheduler(self, optimizer):
        return torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30, eta_min=1e-6)
    
    def train_dataloader(self):
        train = torch.utils.data.TensorDataset(
            torch.tensor(self.X_train, dtype=torch.float32),
            torch.tensor(self.y_train.reshape((len(self.y_train), 1)))
        )    
        train_loader = torch.utils.data.DataLoader(train, batch_size=self.batch_size, shuffle=True,num_workers=0, pin_memory=True)

        return train_loader

    def valid_dataloader(self):
        valid = torch.utils.data.TensorDataset(
            torch.tensor(self.X_valid, dtype=torch.float32),
            torch.tensor(self.y_valid.reshape((len(self.y_valid), 1)))
        )    
        valid_loader = torch.utils.data.DataLoader(valid,
                                                   #batch_size=self.batch_size,
                                                   batch_size=self.batch_size * 2,
                                                   shuffle=False, num_workers=0, pin_memory=True)
        
        return valid_loader

    def test_dataloader(self):
        test = torch.utils.data.TensorDataset(
            torch.tensor(self.X_test, dtype=torch.float32)
        )    
        test_loader = torch.utils.data.DataLoader(test, batch_size=self.batch_size, shuffle=False, num_workers=0, pin_memory=True)

        return test_loader

    def fit(self, patience=10):
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        print(f'使用デバイス: {device}')
        self.net.to(device)
        
        metrics = {'loss': [], 'RMSE': []}
        early_stopping = EarlyStopping(patience=patience, verbose=True)
        optimizer = self.optimizer()
        criterion = self.criterion()
        scheduler = self.scheduler(optimizer)

        for epoch in range(self.num_epochs):
            if (epoch+1) % 10 == 0:
                print(f'Epoch {epoch+1}/{self.num_epochs}')
                print('-'*10)


            # epochごとの学習と検証のループ
            for phase in ['train', 'valid']:
                if phase == 'train':
                    self.net.train()
                else:
                    self.net.eval()

                epoch_loss = 0.0
                valid_trues = np.array([])
                valid_preds = []

                # epoch=0の訓練は省略
                if (epoch==0) and (phase=='train'):
                    continue

                # dataloaderからミニバッチを取り出すループ
                for inputs, labels in self.dataloaders_dict[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    lables = labels.long()
                    # optimの初期化
                    optimizer.zero_grad()

                    # forward
                    with torch.set_grad_enabled(phase=='train'):
                        outputs = self.net(inputs.float())
                        #outputs = outputs.reshape(-1)
                        self.outputs = outputs
                        self.labels = labels
                        loss = criterion(outputs.double(), labels.double()) # 損失の計算
                        preds = outputs.cpu().detach().numpy()

                        # バックプロパゲーション
                        if phase=='train':
                            loss.backward()
                            optimizer.step()
                            

                        # 結果の計算
                        epoch_loss += loss.item() * inputs.size(0)
                        # validのpredを保存
                        valid_trues = np.concatenate((valid_trues, labels.reshape(-1).cpu().detach().numpy()))
                        valid_preds.append(preds)

                # epochごとのlossとaccを表示
                valid_preds = np.concatenate(valid_preds).reshape(-1, self.num_labels)
                epoch_loss = epoch_loss / len(self.dataloaders_dict[phase].dataset)
                self.valid_trues = valid_trues
                self.valid_preds = valid_preds
                epoch_rmse = np.sqrt(mean_squared_error(valid_trues, valid_preds))

                if (epoch+1) % 1 == 0:
                    logger.info(f'{phase} Loss: {epoch_loss:.4f} RMSE: {epoch_rmse:.4f}')
                metrics['loss'].append(epoch_loss)
                metrics['RMSE'].append(epoch_rmse)
                
                if phase == 'train':
                    scheduler.step()
                if phase == 'valid':
                    early_stopping(epoch_loss, self.net)
        
                if early_stopping.early_stop:
                    logger.info("Early stopping")
                    # load the last checkpoint with the best model
                    self.net.load_state_dict(torch.load('checkpoint.pt'))
                    self.metrics = metrics
                    return self.net

        self.metrics = metrics
        return self.net

    def predict(self, use_dataloader='valid'):
        if use_dataloader == 'valid':
            dataloader = self.valid_dataloader
        elif use_dataloader == 'test':
            dataloader = self.test_dataloader
        else:
            return 'use_dataloader is only "valid" or "test"'
        
        self.net.eval()
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        y_pred = []
        for batch in dataloader:
            inputs = batch[0].to(device)
            outputs = self.net(inputs.float())
            preds = outputs.cpu().detach().numpy()
            y_pred.append(preds)
                
        y_pred = np.concatenate(y_pred).reshape(-1, self.num_labels)
        return y_pred


In [7]:
seed = 7188

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
#os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
logger.info('Set random seeds')

[I 201215 17:58:10 <ipython-input-7-54b34ea31ce8>:10] Set random seeds


In [8]:
# scaling
X = pd.concat([X_train, X_test], axis=0, ignore_index=True)
scaler = StandardScaler()
X = scaler.fit_transform(X.fillna(-1))
X_train = pd.DataFrame(X[:X_train.shape[0], :], columns=X_train.columns)
X_test = pd.DataFrame(X[X_train.shape[0]:, :], columns=X_train.columns)
logger.info(f'X_train shapes: {X_train.shape}')
logger.info(f'X_test shapes: {X_test.shape}')

[I 201215 17:58:10 <ipython-input-8-f615581bf5a2>:7] X_train shapes: (7653, 639)
[I 201215 17:58:10 <ipython-input-8-f615581bf5a2>:8] X_test shapes: (8360, 639)


In [9]:
#RankGauss
#transformer = QuantileTransformer(n_quantiles=1000, random_state=0, output_distribution='normal')
#transformer.fit(X_train.fillna(-1))
#
#X_train.iloc[:, :] = transformer.transform(X_train.fillna(-1))
#X_test.iloc[:, :] = transformer.transform(X_test.fillna(-1))

In [10]:
oof_df = pd.DataFrame()
y_pred_df = pd.DataFrame()
#'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 
for objective in tqdm(['Global_Sales']):
    y_train = np.log1p(train[objective])
    
    stratified_y = pd.cut(y_train, 10, labels=False)
    
    #kf = GroupKFold(n_splits=5)
    kf = StratifiedGroupKFold(n_splits=5, random_state=0)
    oof = np.zeros(X_train.shape[0])
    y_pred = np.zeros(X_test.shape[0])
    models = []

    for i, (train_index, valid_index) in enumerate(kf.split(X_train, y=stratified_y, groups=groups)):
        print(f'Start {i+1} fold')
        print('-' * 20)
        X_tr, y_tr = X_train.iloc[train_index, :], y_train.iloc[train_index]
        X_val, y_val = X_train.iloc[valid_index, :], y_train.iloc[valid_index]
        X_tes = X_test.copy()

        net = NetRegressor(in_features=X_train.shape[1], out_features=1)
        model = MLPRegressor(net, X_tr, y_tr, X_val, y_val, X_tes, num_labels=1, batch_size=512, num_epochs=1000)
        model.fit() 
        oof[valid_index] = model.predict('valid').reshape(-1)
        y_pred += model.predict('test').reshape(-1) / 5
        models.append(model)
        
    oof = np.where(oof < 0, 0, oof)
    y_pred = np.where(y_pred < 0, 0, y_pred)
    oof = np.expm1(oof)
    y_pred = np.expm1(y_pred)
    oof_df[objective] = oof
    y_pred_df[objective] = y_pred

Start 1 fold
--------------------
使用デバイス: cuda:0


[I 201215 17:58:17 <ipython-input-6-edd6cd26e1c9>:200] valid Loss: 9.8446 RMSE: 3.1376
[I 201215 17:58:17 <ipython-input-6-edd6cd26e1c9>:65] Validation loss decreased (inf --> 9.844596).  Saving model ...
[I 201215 17:58:17 <ipython-input-6-edd6cd26e1c9>:200] train Loss: 8.8256 RMSE: 2.9708
[I 201215 17:58:17 <ipython-input-6-edd6cd26e1c9>:200] valid Loss: 6.4050 RMSE: 2.5308
[I 201215 17:58:17 <ipython-input-6-edd6cd26e1c9>:65] Validation loss decreased (9.844596 --> 6.405044).  Saving model ...
[I 201215 17:58:17 <ipython-input-6-edd6cd26e1c9>:200] train Loss: 6.0253 RMSE: 2.4546
[I 201215 17:58:17 <ipython-input-6-edd6cd26e1c9>:200] valid Loss: 4.0537 RMSE: 2.0134
[I 201215 17:58:17 <ipython-input-6-edd6cd26e1c9>:65] Validation loss decreased (6.405044 --> 4.053730).  Saving model ...
[I 201215 17:58:17 <ipython-input-6-edd6cd26e1c9>:200] train Loss: 4.2918 RMSE: 2.0717
[I 201215 17:58:17 <ipython-input-6-edd6cd26e1c9>:200] valid Loss: 2.7406 RMSE: 1.6555
[I 201215 17:58:17 <ipython

Epoch 10/1000
----------


[I 201215 17:58:18 <ipython-input-6-edd6cd26e1c9>:200] train Loss: 0.8695 RMSE: 0.9325
[I 201215 17:58:18 <ipython-input-6-edd6cd26e1c9>:200] valid Loss: 0.9060 RMSE: 0.9519
[I 201215 17:58:18 <ipython-input-6-edd6cd26e1c9>:65] Validation loss decreased (1.015702 --> 0.906037).  Saving model ...
[I 201215 17:58:18 <ipython-input-6-edd6cd26e1c9>:200] train Loss: 0.8356 RMSE: 0.9141
[I 201215 17:58:18 <ipython-input-6-edd6cd26e1c9>:200] valid Loss: 0.8786 RMSE: 0.9374
[I 201215 17:58:18 <ipython-input-6-edd6cd26e1c9>:65] Validation loss decreased (0.906037 --> 0.878637).  Saving model ...
[I 201215 17:58:18 <ipython-input-6-edd6cd26e1c9>:200] train Loss: 0.8002 RMSE: 0.8946
[I 201215 17:58:18 <ipython-input-6-edd6cd26e1c9>:200] valid Loss: 0.8965 RMSE: 0.9468
[I 201215 17:58:18 <ipython-input-6-edd6cd26e1c9>:54] EarlyStopping counter: 1 out of 10
[I 201215 17:58:18 <ipython-input-6-edd6cd26e1c9>:200] train Loss: 0.7809 RMSE: 0.8837
[I 201215 17:58:18 <ipython-input-6-edd6cd26e1c9>:200] v

Epoch 20/1000
----------


...

[I 201215 17:58:43 <ipython-input-6-edd6cd26e1c9>:200] train Loss: 0.7968 RMSE: 0.8927
[I 201215 17:58:43 <ipython-input-6-edd6cd26e1c9>:200] valid Loss: 0.8360 RMSE: 0.9143
[I 201215 17:58:43 <ipython-input-6-edd6cd26e1c9>:65] Validation loss decreased (0.879597 --> 0.836018).  Saving model ...
[I 201215 17:58:43 <ipython-input-6-edd6cd26e1c9>:200] train Loss: 0.7707 RMSE: 0.8779
[I 201215 17:58:43 <ipython-input-6-edd6cd26e1c9>:200] valid Loss: 0.8279 RMSE: 0.9099
[I 201215 17:58:43 <ipython-input-6-edd6cd26e1c9>:65] Validation loss decreased (0.836018 --> 0.827939).  Saving model ...
[I 201215 17:58:43 <ipython-input-6-edd6cd26e1c9>:200] train Loss: 0.7581 RMSE: 0.8707
[I 201215 17:58:43 <ipython-input-6-edd6cd26e1c9>:200] valid Loss: 0.7932 RMSE: 0.8906
[I 201215 17:58:43 <ipython-input-6-edd6cd26e1c9>:65] Validation loss decreased (0.827939 --> 0.793232).  Saving model ...
[I 201215 17:58:43 <ipython-input-6-edd6cd26e1c9>:200] train Loss: 0.7317 RMSE: 0.8554
[I 201215 17:58:43 <ip

Epoch 20/1000
----------


[I 201215 17:58:44 <ipython-input-6-edd6cd26e1c9>:200] train Loss: 0.6307 RMSE: 0.7942
[I 201215 17:58:44 <ipython-input-6-edd6cd26e1c9>:200] valid Loss: 0.7334 RMSE: 0.8564
[I 201215 17:58:44 <ipython-input-6-edd6cd26e1c9>:65] Validation loss decreased (0.751938 --> 0.733365).  Saving model ...
[I 201215 17:58:44 <ipython-input-6-edd6cd26e1c9>:200] train Loss: 0.6373 RMSE: 0.7983
[I 201215 17:58:44 <ipython-input-6-edd6cd26e1c9>:200] valid Loss: 0.7366 RMSE: 0.8583
[I 201215 17:58:44 <ipython-input-6-edd6cd26e1c9>:54] EarlyStopping counter: 1 out of 10
[I 201215 17:58:44 <ipython-input-6-edd6cd26e1c9>:200] train Loss: 0.6355 RMSE: 0.7972
[I 201215 17:58:44 <ipython-input-6-edd6cd26e1c9>:200] valid Loss: 0.7443 RMSE: 0.8627
[I 201215 17:58:44 <ipython-input-6-edd6cd26e1c9>:54] EarlyStopping counter: 2 out of 10
[I 201215 17:58:44 <ipython-input-6-edd6cd26e1c9>:200] train Loss: 0.6313 RMSE: 0.7945
[I 201215 17:58:44 <ipython-input-6-edd6cd26e1c9>:200] valid Loss: 0.7492 RMSE: 0.8655
[I 

Epoch 30/1000
----------


[I 201215 17:58:45 <ipython-input-6-edd6cd26e1c9>:200] train Loss: 0.6140 RMSE: 0.7836
[I 201215 17:58:45 <ipython-input-6-edd6cd26e1c9>:200] valid Loss: 0.7405 RMSE: 0.8605
[I 201215 17:58:45 <ipython-input-6-edd6cd26e1c9>:54] EarlyStopping counter: 10 out of 10
[I 201215 17:58:45 <ipython-input-6-edd6cd26e1c9>:210] Early stopping


In [11]:
joblib.dump((oof_df['Global_Sales'].values, y_pred_df['Global_Sales'].values), '../output/MLP_oof_preds.joblib')

['../output/MLP_oof_preds.joblib']

In [12]:
oof = oof_df['Global_Sales'].values

oof = np.where(oof < 0, 0, oof)
#y_pred = np.where(y_pred < 0, 0, y_pred)

y_train = np.expm1(y_train)
#oof = np.expm1(oof)
#y_pred = np.expm1(y_pred)

print(mean_squared_log_error(y_train, oof) ** .5)

0.8966913673892108
